In [19]:
import pygplates
import numpy as np
import collections

file_template = '/Users/simon/Data/GPlates/PublishedModels/Boschman_Andes/2.reconstruction input/shapes/all/reconstructed_{:0.2f}Ma.shp'

global_rotation_model = pygplates.RotationModel('/Users/simon/Data/GPlates/PublishedModels/vanHinsbergen_GPlates_reconstructions/190701_master.rot')

# TODO
# Also include the 201-0 rotations ?


In [2]:
present_day_features = pygplates.FeatureCollection(file_template.format(0))

reconstructed_features_dict = {}
for reconstruction_time in np.arange(1,81,1):
    reconstructed_features_dict[reconstruction_time] = pygplates.FeatureCollection(file_template.format(reconstruction_time))
    
reconstructed_features_dict = collections.OrderedDict(sorted(reconstructed_features_dict.items()))


In [23]:
anchor_plate_id = 0

rotation_features = []
completed_plate_ids = []


######################################################################
# First the rotations from the global model for 201 relative to 000
pole_list = []

for reconstruction_time in np.arange(0,81,1):
    finite_rotation = global_rotation_model.get_rotation(reconstruction_time, 201, 0, 0)
    pole_list.append(pygplates.GpmlTimeSample(
            pygplates.GpmlFiniteRotation(finite_rotation),
            reconstruction_time,
            'Reverse-Engineered Rotation from Shapefiles')
                    )
    
rotation_features.append(
    pygplates.Feature.create_total_reconstruction_sequence(
        anchor_plate_id,
        feature_plate_id,
        pygplates.GpmlIrregularSampling(pole_list),
        name='Hack')
)


######################################################################
# Now the polygons within the Andes
for f in present_day_features:
    
    present_day_geometry = f.get_geometry()
    feature_id = f.get_feature_id()
    feature_plate_id = f.get_reconstruction_plate_id()
    
    if feature_plate_id in completed_plate_ids:
        continue
    print(feature_plate_id)
    
    
    pole_list = []
    pole_list.append(pygplates.GpmlTimeSample(
            pygplates.GpmlFiniteRotation(pygplates.FiniteRotation.create_identity_rotation()),
            0,
            'Reverse-Engineered Rotation from Shapefiles')
                        )
    
    for reconstruction_time in np.arange(1,81,1):

        reconstructed_feature = reconstructed_features_dict[reconstruction_time].get(
            lambda feature: feature.get_feature_id()==f.get_feature_id(),
            pygplates.FeatureReturn.all)

        if len(reconstructed_feature)==0:
            break
        #print(reconstructed_feature)
        reconstructed_geometry = reconstructed_feature[0].get_geometry()
        
        finite_rotation = pygplates.FiniteRotation.create_segment_rotation(present_day_geometry[0], present_day_geometry[1],
                                                                           reconstructed_geometry[0], reconstructed_geometry[1])
        
        #print(finite_rotation)
        
        pole_list.append(pygplates.GpmlTimeSample(
            pygplates.GpmlFiniteRotation(finite_rotation),
            reconstruction_time,
            'Reverse-Engineered Rotation from Shapefiles')
                        )
        
    rotation_features.append(
        pygplates.Feature.create_total_reconstruction_sequence(
            anchor_plate_id,
            feature_plate_id,
            pygplates.GpmlIrregularSampling(pole_list),
            name='Hack')
    )
    
    completed_plate_ids.append(feature_plate_id)
    
    #break

pygplates.FeatureCollection(rotation_features).write('hack_rotations.rot')

2598
2526
2595
2573
2527
2515
2596
25961
25412
25411
25251
25221
2537
2530
2529
2525
2522
2541
2597
201
503
469
513
515
511
316
512
405
407
410
409
350
351
360
352
358
337
353
354
359
453
440
441
400
2549
2550
2555
25471
2547
2516
2592
2504
25181
2513
2518
25051
2512
2503
2578
2508
2502
2583
